In [1]:
import pandas as pd

from pandas_datareader import data as pdr
import yfinance as yf
import streamlit as st
import numpy as np

from statsmodels.tsa.stattools import acf
from scipy.stats import wilcoxon, hmean

from tqdm.notebook import tqdm
from multiprocess import Pool
import multiprocessing

from matplotlib import pyplot as plt
import seaborn as sb

import warnings
warnings.filterwarnings('ignore', '.*distplot.*', )

from optimal_weights import random_portfolio, min_var_portfolio, max_snr_portfolio
from evaluate_weights import evaluate_weights

# Download historical prices (adjusted close)

In [2]:
symbols=['AAPL', 'NKE', 'GOOGL', 'AMZN']
start = '2005-01-01' # expected datetime format is '%Y-%m-%d'
end = '2022-12-31'   # expected datetime format is '%Y-%m-%d'

In [3]:
yf.pdr_override()
price_df = pd.DataFrame()

for symbol in symbols:
    user_input = st.text_input('Enter Stock Ticker', symbol)
    tmp_df = pdr.get_data_yahoo(user_input, start=start, end=end)
    price_df = pd.concat([price_df, tmp_df['Adj Close']], axis='columns')

price_df.columns = symbols
price_df.head()

2023-01-09 11:16:27.357 
  command:

    streamlit run c:\Users\edmzlat\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,AAPL,NKE,GOOGL,AMZN
2005-01-03 00:00:00,0.963385,9.136415,5.072823,2.2260
2005-01-04 00:00:00,0.973278,9.006457,4.867367,2.1070
2005-01-05 00:00:00,0.981803,8.898663,4.842593,2.0885
2005-01-06 00:00:00,0.982564,8.887582,4.718468,2.0525
2005-01-07 00:00:00,1.054106,8.835194,4.851101,2.1160


In [4]:
price_df.pct_change().describe()

,AAPL,NKE,GOOGL,AMZN
count,4530.000000,4530.000000,4530.000000,4530.000000
mean,0.001301,0.000725,0.000809,0.001095
std,0.020858,0.018051,0.018956,0.024336
min,-0.179195,-0.128081,-0.116341,-0.218220
25%,-0.008611,-0.007687,-0.007847,-0.009981
50%,0.001001,0.000577,0.000690,0.000560
75%,0.011993,0.009021,0.009770,0.012229
max,0.139049,0.155315,0.199915,0.269497


# Take independent subsamples (windows) from the data and generate portfolios

In [5]:
def get_largest_significant_lags(df, alfa=0.05):
    largest_significant_lag = 0
    
    for i in range(df.shape[1]):
        significant_lags = np.where(acf(df.iloc[:, i], missing="conservative", qstat=True)[-1] < alfa)[0]

        if significant_lags.shape[0] == 0:
            largest_significant_lag = max(largest_significant_lag, 0)
        else:
            # zero lag is ignored in the LB-test, so the index must be shifted by 1
            largest_significant_lag = max(largest_significant_lag, significant_lags[-1] + 1)
    
    return largest_significant_lag

In [7]:
window_size = 100
num_portfolios = 10000
max_pool = multiprocessing.cpu_count()


window_start = 0
window_end = window_size
largest_significant_lag = 0
num_assets = len(price_df.columns)

results = []

with tqdm(total=price_df.shape[0]) as pbar:

    while window_start + window_size < price_df.shape[0]:
        window_df = price_df.iloc[window_start:window_end, :]
        
        # find mean and covariance of returns
        cov_matrix = window_df.pct_change().cov()
        exp_returns = window_df.pct_change().mean()
        
        cumul_returns_df = window_df / window_df.iloc[0,:]
        
        for longonly in [True, False]:
        
            w_min_var = min_var_portfolio(cov_matrix, longonly=longonly)
            w_max_snr = max_snr_portfolio(cov_matrix, exp_returns, longonly=longonly)

            # find random portfolio Hurst exponent(s)
            with Pool(max_pool) as p:
                pool_outputs = list(
                    tqdm(
                        p.imap(
                            random_portfolio,
                            [(seed, num_assets, cumul_returns_df, longonly) for seed in range(num_portfolios)]
                        ),
                        total=num_portfolios
                    )
                )

            results.append(
                evaluate_weights(w_min_var, cumul_returns_df) +
                evaluate_weights(w_max_snr, cumul_returns_df) +
                [ sum(x)/float(len(pool_outputs)) for x in zip(*pool_outputs) ] +
                [longonly]
            )
        
        ### move window 
        largest_significant_lag = get_largest_significant_lags(window_df.pct_change(), alfa=0.05)
        
        window_start = window_end + largest_significant_lag
        window_end = window_start + window_size

        pbar.update(window_start)

  0%|          | 0/4531 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
results_df = pd.DataFrame(results, columns=["H1_minvar", "H2_minvar", "H3_minvar", "H4_minvar", "H1_maxsnr", "H2_maxsnr", "H3_maxsnr", "H4_maxsnr", "H1_random", "H2_random", "H3_random", "H4_random", "longonly"])
results_df.to_csv("results_df.csv")
results_df

# Analyze portfolios

In [ ]:
results_df = pd.read_csv("results_df.csv", index_col=0)
results_df

In [ ]:
portfolios = ["maxsnr", "minvar", "random"]

# H1: rescaled range
# H2: robust variance of lagged difference
# H3: variance of lagged difference
# H4: fractal dimensiong
hurst_estimators = ["H1", "H2", "H3", "H4"]

## Check Hurst estimator distributions

In [ ]:
longonly = results_df.longonly == True

fig, axes = plt.subplots(2, 2, figsize=(20, 14))
fig.suptitle("Distributions of the different Hurst estimators", fontsize=20)

for j in range(len(hurst_estimators)):
    for i in range(len(portfolios)):
        ax = axes[j // 2][j % 2]
        sb.distplot(results_df[longonly][hurst_estimators[j]+"_"+portfolios[i]], kde=True, label=portfolios[i], ax=ax)
        ax.set_xlabel(hurst_estimators[j])
        ax.legend(loc="upper right")
        ax.set_xlim(0.0, 1.0)

In [ ]:
mask = results_df.longonly == False

fig, axes = plt.subplots(2, 2, figsize=(20, 14))
fig.suptitle("Distributions of the different Hurst estimators", fontsize=20)

for j in range(len(hurst_estimators)):
    for i in range(len(portfolios)):
        ax = axes[j // 2][j % 2]
        sb.distplot(results_df[~longonly][hurst_estimators[j]+"_"+portfolios[i]], kde=True, label=portfolios[i], ax=ax)
        ax.set_xlabel(hurst_estimators[j])
        ax.legend(loc="upper right")
        ax.set_xlim(0.0, 1.0)

## Hypothesis test: equivalence of means

In [ ]:
# H0: sample means are equal
# H1: the first mean is larger

In [ ]:
index = []
data = []

# Test equivalence
cntr = 0
for i in range(len(portfolios)):
    for j in range(i+1, len(portfolios)):
        index.append((cntr, portfolios[i], "mean"))
        data.append([results_df[longonly][hurst_estimator+"_"+portfolios[i]].mean() for hurst_estimator in hurst_estimators] + [""])
        
        index.append((cntr, portfolios[j], "mean"))
        data.append([results_df[longonly][hurst_estimator+"_"+portfolios[j]].mean() for hurst_estimator in hurst_estimators] + [""])
        
        index.append((cntr, "Wilcoxon", "T"))
        index.append((cntr, "Wilcoxon", "p"))
        
        data.append([wilcoxon(results_df[longonly][hurst_estimator+"_"+portfolios[i]], results_df[longonly][hurst_estimator+"_"+portfolios[j]], alternative="greater")[0] for hurst_estimator in hurst_estimators] + [""])
        data.append([wilcoxon(results_df[longonly][hurst_estimator+"_"+portfolios[i]], results_df[longonly][hurst_estimator+"_"+portfolios[j]], alternative="greater")[1] for hurst_estimator in hurst_estimators])
        data[-1].append(hmean(data[-1]))
        
        cntr += 1

pd.DataFrame(data, index=pd.MultiIndex.from_tuples(index), columns=hurst_estimators+["hmp"])

In [ ]:
index = []
data = []

# Test equivalence
cntr = 0
for i in range(len(portfolios)):
    for j in range(i+1, len(portfolios)):
        index.append((cntr, portfolios[i], "mean"))
        data.append([results_df[~longonly][hurst_estimator+"_"+portfolios[i]].mean() for hurst_estimator in hurst_estimators] + [""])
        
        index.append((cntr, portfolios[j], "mean"))
        data.append([results_df[~longonly][hurst_estimator+"_"+portfolios[j]].mean() for hurst_estimator in hurst_estimators] + [""])
        
        index.append((cntr, "Wilcoxon", "T"))
        index.append((cntr, "Wilcoxon", "p"))
        
        data.append([wilcoxon(results_df[~longonly][hurst_estimator+"_"+portfolios[i]], results_df[~longonly][hurst_estimator+"_"+portfolios[j]], alternative="greater")[0] for hurst_estimator in hurst_estimators] + [""])
        data.append([wilcoxon(results_df[~longonly][hurst_estimator+"_"+portfolios[i]], results_df[~longonly][hurst_estimator+"_"+portfolios[j]], alternative="greater")[1] for hurst_estimator in hurst_estimators])
        data[-1].append(hmean(data[-1]))
        
        cntr += 1

pd.DataFrame(data, index=pd.MultiIndex.from_tuples(index), columns=hurst_estimators+["hmp"])